In [5]:
#
# Mountaincar problem using DQN and seperate target network
#

import gym
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from collections import deque


class DQN:
    def __init__(self, env):
        # Environment to use
        self.env = env
        # Replay memory
        self.memory = deque(maxlen=10000)

        # Discount factor
        self.gamma = 0.99

        # Initial exploration factor
        self.epsilon = 1.0
        # Minimum value exploration factor
        self.epsilon_min = 0.005
        # Decay for epsilon
        self.epsilon_decay = (self.epsilon - self.epsilon_min) / 50000

        self.batch_size = 64
        self.train_start = 1000
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n

        # Learning rate
        self.learning_rate = 0.001

        # Model being trained
        self.model = self.create_model()
        # Target model used to predict Q(S,A)
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()
        model.add(Dense(
            32, input_dim=self.state_size, activation='relu', kernel_initializer="he_uniform"))
        model.add(Dense(16, activation='relu', kernel_initializer="he_uniform"))
        model.add(Dense(self.env.action_space.n, activation="linear",
                        kernel_initializer="he_uniform"))
        model.compile(
            loss="mean_squared_error", optimizer=Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        # Decay exploration rate by epsilon decay
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state)[0])

    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        if len(self.memory) < self.train_start:
            return

        mini_batch = random.sample(self.memory, self.batch_size)

        update_input = np.zeros((self.batch_size, self.state_size))
        update_target = np.zeros((self.batch_size, self.action_size))

        for i in range(self.batch_size):
            state, action, reward, next_state, done = mini_batch[i]
            target = self.model.predict(state)[0]

            if done:
                target[action] = reward
            else:
                target[action] = reward + self.gamma * \
                    np.amax(self.target_model.predict(next_state)[0])
            update_input[i] = state
            update_target[i] = target
        
        #print("update_input: {}, update_target: {}".format(update_input,update_target))
        self.model.fit(update_input, update_target,
                       batch_size=self.batch_size, epochs=1, verbose=0)

    def target_train(self):
        # Simply copy the weights of the model to target_model
        self.target_model.set_weights(self.model.get_weights())
        return

    def save_model(self, fn):
        self.model.save(fn)


def main():
    env = gym.make("Acrobot-v1")

    trials = 4000
    trial_len = 500

    dqn_agent = DQN(env=env)
    for trial in range(trials):
        cur_state = env.reset().reshape(1, 6)
        #print("{}" .format(cur_state))
        for step in range(trial_len):
            action = dqn_agent.act(cur_state)
            new_state, reward, done, _ = env.step(action)
            #print("{}" .format(new_state))
            new_state = new_state.reshape(1, 6)
            #print("{}" .format(new_state))
            dqn_agent.remember(cur_state, action, reward, new_state, done)

            dqn_agent.replay()

            cur_state = new_state
            if done:
                env.reset()
                dqn_agent.target_train()
                break

        print("Iteration: {} Score: -{}".format(trial, step))


if __name__ == "__main__":
    main()


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Iteration: 0 Score: -499
Iteration: 1 Score: -499
Iteration: 2 Score: -499
Iteration: 3 Score: -499
Iteration: 4 Score: -499
Iteration: 5 Score: -499
Iteration: 6 Score: -499
Iteration: 7 Score: -499
Iteration: 8 Score: -499
Iteration: 9 Score: -499
Iteration: 10 Score: -499
Iteration: 11 Score: -499
Iteration: 12 Score: -499
Iteration: 13 Score: -486
Iteration: 14 Score: -376
Iteration: 15 Score: -499
Iteration: 16 Score: -499
Iteration: 17 Score: -499
Iteration: 18 Score: -499
Iteration: 19 Score: -499
Iteration: 20 Score: -499
Iteration: 21 Score: -417
Iteration: 22 Score: -499
Iteration: 23 Score: -499
Iteration: 24 Score: -461
Iteration: 25 Score: -499
Iteration: 26 Score: -385
Iteration: 27 Score: -408
Iteration: 28 Score: -471
Iteration: 29 Score: -377
Iteration: 30 Score: -233
Iteration: 31 Score: -499
Iteration: 32 Score: -471
Iteration: 33 Score: -317
Iteration: 34 Score: -345
I

KeyboardInterrupt: 